In [1]:
from utils.Data_by_API import *

In [17]:
import urllib.request
from bs4 import BeautifulSoup

In [27]:
import xml.etree.ElementTree as ET

In [26]:
from xml.etree.ElementTree import parse

In [39]:
xtree  = ET.fromstring(rq.text)

In [40]:
xmlsoup = BeautifulSoup(rq.text,'html.parser')
items = xmlsoup.find_all('item')

In [ ]:
item.title.get_text()
item.description.get_text()

In [46]:
items[0].datename.get_text()

'추석'

In [43]:
items[0].title.get_text()

AttributeError: 'NoneType' object has no attribute 'get_text'

In [22]:
response  = urllib.request.urlopen(url)
rescode = response.getcode()

In [25]:
response.read().decode('utf-8')

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><dateKind>01</dateKind><dateName>추석</dateName><isHoliday>Y</isHoliday><locdate>20150926</locdate><seq>1</seq></item><item><dateKind>01</dateKind><dateName>추석</dateName><isHoliday>Y</isHoliday><locdate>20150927</locdate><seq>1</seq></item><item><dateKind>01</dateKind><dateName>추석</dateName><isHoliday>Y</isHoliday><locdate>20150928</locdate><seq>1</seq></item><item><dateKind>01</dateKind><dateName>대체공휴일</dateName><isHoliday>Y</isHoliday><locdate>20150929</locdate><seq>1</seq></item></items><numOfRows>10</numOfRows><pageNo>1</pageNo><totalCount>4</totalCount></body></response>'

In [2]:
url = "http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getHoliDeInfo?serviceKey=eLWdQyzctRdtv8bEOuewsTtK6sNkoWp1bE74OUBk43jg4tU6AsI6yYt6Z%2B7sOeaqtB5pTH2yHuPRIuEHtu5amQ%3D%3D&solYear=2015&solMonth=09"

In [6]:
api = Data_by_API(url, params_dict = {})

In [7]:
api.create_request_url()

In [10]:
api.request_url

'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getHoliDeInfo?serviceKey=eLWdQyzctRdtv8bEOuewsTtK6sNkoWp1bE74OUBk43jg4tU6AsI6yYt6Z%2B7sOeaqtB5pTH2yHuPRIuEHtu5amQ%3D%3D&solYear=2015&solMonth=09'

In [12]:
rq = api.request()

In [29]:
rq.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><dateKind>01</dateKind><dateName>추석</dateName><isHoliday>Y</isHoliday><locdate>20150926</locdate><seq>1</seq></item><item><dateKind>01</dateKind><dateName>추석</dateName><isHoliday>Y</isHoliday><locdate>20150927</locdate><seq>1</seq></item><item><dateKind>01</dateKind><dateName>추석</dateName><isHoliday>Y</isHoliday><locdate>20150928</locdate><seq>1</seq></item><item><dateKind>01</dateKind><dateName>대체공휴일</dateName><isHoliday>Y</isHoliday><locdate>20150929</locdate><seq>1</seq></item></items><numOfRows>10</numOfRows><pageNo>1</pageNo><totalCount>4</totalCount></body></response>'

In [ ]:
rq = request()

In [178]:
import requests

class Data_by_API(object):
    
    def __init__(self, url):
        self.url = url
#         self.serviceKey = serviceKey
        
    def create_request_url(self, params_dict):
#         params_dict["service_key"] = self.serviceKey
        params_list = [f"{k}={v}" for k, v in params_dict.items()]
        params_str = "&".join(params_list)
#         print(params_str)
        
        self.request_url = self.url + params_str
        
        return self.request_url
    
    
    def request(self, request_url = None):
        
        if request_url == None:
            request_url = self.request_url
            
        rq = requests.get(request_url)
        
        return rq
        

In [259]:
import pandas as pd
from collections import defaultdict

class Holiday_Data_by_API(Data_by_API):
    
    holiday_url = "http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getHoliDeInfo?"
    restday_url = "http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?"
    
    def __init__(self, params_dict, type):
        if type == "holi":
            base_url = self.holiday_url
        elif type == "rest":
            base_url = self.restday_url
            
        super().__init__(url = base_url)
#         self.year = year
        self.params_dict = params_dict
    
    def create_request_urls(self, params_dict):
        params_dict = params_dict.copy()
        request_urls = []
        for x in range(1, 13):
            if x < 10:
                params_dict["solMonth"] =  f"0{str(x)}"

            else:
                params_dict["solMonth"] =  str(x)
                
            request_urls.append(self.create_request_url(params_dict = params_dict))
    
        return request_urls
    
    
    def get(self):
        
        if "solMonth" in self.params_dict.keys():
            self.request_urls = [self.create_request_url(params_dict = self.params_dict)]
        else:             
            self.request_urls = self.create_request_urls(params_dict = self.params_dict)
        
        data_dict = defaultdict(list)
        for request_url in self.request_urls:
            rq = super().request(request_url = request_url)
#             print(rq.text)
            xmlsoup = BeautifulSoup(rq.text,'html.parser')
            items = xmlsoup.find_all('item')
#             print(items)
            
            for item in items:
                data_dict["locdate"].append(item.locdate.get_text())
                data_dict["datename"].append(item.datename.get_text())
                    
        return pd.DataFrame(data_dict)

In [261]:
temp = Holiday_Data_by_API(params_dict = {"serviceKey" : "eLWdQyzctRdtv8bEOuewsTtK6sNkoWp1bE74OUBk43jg4tU6AsI6yYt6Z%2B7sOeaqtB5pTH2yHuPRIuEHtu5amQ%3D%3D",
                                          "solYear" : "2020"},
                           type = "holi")

In [263]:
temp = Holiday_Data_by_API(params_dict = {"serviceKey" : "eLWdQyzctRdtv8bEOuewsTtK6sNkoWp1bE74OUBk43jg4tU6AsI6yYt6Z%2B7sOeaqtB5pTH2yHuPRIuEHtu5amQ%3D%3D",
                                          "solYear" : "2020"},
                           type = "rest")

In [262]:
temp.get()

,locdate,datename
0,20200101,1월1일
1,20200124,설날
2,20200125,설날
3,20200126,설날
4,20200127,설날
5,20200301,삼일절
6,20200415,제21대 국회의원선거
7,20200430,부처님오신날
8,20200505,어린이날
9,20200606,현충일


In [238]:
params_dict = {"serviceKey" : "eLWdQyzctRdtv8bEOuewsTtK6sNkoWp1bE74OUBk43jg4tU6AsI6yYt6Z%2B7sOeaqtB5pTH2yHuPRIuEHtu5amQ%3D%3D",
                                          "solYear" : "2020"}

In [240]:
aa = params_dict.copy()

In [241]:
aa["asdfs"] = "123123321"